# Running Docker from Python

In [1]:
import docker

In [2]:
client = docker.from_env()

In [3]:
# hello world
client.containers.run("hello-world")

b'\nHello from Docker!\nThis message shows that your installation appears to be working correctly.\n\nTo generate this message, Docker took the following steps:\n 1. The Docker client contacted the Docker daemon.\n 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.\n    (amd64)\n 3. The Docker daemon created a new container from that image which runs the\n    executable that produces the output you are currently reading.\n 4. The Docker daemon streamed that output to the Docker client, which sent it\n    to your terminal.\n\nTo try something more ambitious, you can run an Ubuntu container with:\n $ docker run -it ubuntu bash\n\nShare images, automate workflows, and more with a free Docker ID:\n https://cloud.docker.com/\n\nFor more examples and ideas, visit:\n https://docs.docker.com/engine/userguide/\n\n'

In [4]:
with open('/opt/workdir/test/dockerfile','r') as f:
    for row in f:
        print(row.strip())

FROM ubuntu:16.04
MAINTAINER cvusk

RUN echo "test python client for docker" >> /tmp/test.log

CMD ["cat", "/tmp/test.log"]


In [5]:
# build test image
client.images.build(path="/opt/workdir/test", tag="ubuntu:test")

(<Image: 'ubuntu:test'>, <itertools._tee at 0x7f467d75b448>)

In [6]:
# run test image
client.containers.run("ubuntu:test")

b'test python client for docker\n'

# Controlling Kubernetes from Python

In [7]:
from kubernetes import client as kclient
from kubernetes import config as kconfig
import yaml
import os

In [8]:
kconfig.load_kube_config()

In [9]:
v1 = kclient.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
10.146.0.4	kube-system	etcd-pdk
10.146.0.4	kube-system	kube-apiserver-pdk
10.146.0.4	kube-system	kube-controller-manager-pdk
10.244.0.3	kube-system	kube-dns-6f4fd4bdf-z7q6k
10.146.0.4	kube-system	kube-flannel-ds-9r8cs
10.146.0.4	kube-system	kube-proxy-x24fx
10.146.0.4	kube-system	kube-scheduler-pdk


In [10]:
nginxdep = {'apiVersion': 'extensions/v1beta1',
            'kind': 'Deployment',
            'metadata': {'name': 'my-nginx'},
            'spec': {'replicas': 1,
                     'selector': {'matchLabels': {'run': 'my-nginx'}},
                     'template': {'metadata': {'labels': {'run': 'my-nginx'}},
                                  'spec': {'containers': [{'image': 'nginx',
                                                           'name': 'my-nginx',
                                                           'ports': [{'containerPort': 80}]}]}}}}
nginxdep

{'apiVersion': 'extensions/v1beta1',
 'kind': 'Deployment',
 'metadata': {'name': 'my-nginx'},
 'spec': {'replicas': 1,
  'selector': {'matchLabels': {'run': 'my-nginx'}},
  'template': {'metadata': {'labels': {'run': 'my-nginx'}},
   'spec': {'containers': [{'image': 'nginx',
      'name': 'my-nginx',
      'ports': [{'containerPort': 80}]}]}}}}

In [11]:
nginxsvc = {'apiVersion': 'v1',
            'kind': 'Service',
            'metadata': {'labels': {'run': 'my-nginx'}, 'name': 'my-nginx'},
            'spec': {'externalIPs': ['35.200.42.99'],
                     'ports': [{'nodePort': 30000, 'port': 80, 'protocol': 'TCP'}],
                     'selector': {'run': 'my-nginx'},
                     'type': 'LoadBalancer'}}
nginxsvc

{'apiVersion': 'v1',
 'kind': 'Service',
 'metadata': {'labels': {'run': 'my-nginx'}, 'name': 'my-nginx'},
 'spec': {'externalIPs': ['35.200.42.99'],
  'ports': [{'nodePort': 30000, 'port': 80, 'protocol': 'TCP'}],
  'selector': {'run': 'my-nginx'},
  'type': 'LoadBalancer'}}

In [12]:
with open("kube/nginxdep.yml", "w") as f:
    f.write(yaml.dump(nginxdep, default_flow_style=False))

In [13]:
with open("kube/nginxsvc.yml", "w") as f:
    f.write(yaml.dump(nginxsvc, default_flow_style=False))

In [14]:
 class KubeDeployer(object):
    def __init__(self):
        kconfig.load_kube_config()
        self.__k8s_beta = kclient.ExtensionsV1beta1Api()
        self.__k8s_core = kclient.CoreV1Api()
    def createDeployment(self, filename, filedir=None):
        if filedir is not None:
            filename = os.path.join(filedir, filename)
        with open(filename) as f:
            dep = yaml.load(f)
            resp = self.__k8s_beta.create_namespaced_deployment(
                body=dep, namespace="default")
            print("DEPLOYMENT {0} created. status={1}".format(filename, resp.metadata))
        return resp
    def getDeploymentList(self):
        resp = self.__k8s_beta.list_deployment_for_all_namespaces()
        return resp
    def deleteDeployment(self, name, namespace="default", **kwargs):
        body = kubernetes.client.V1DeleteOptions(**kwargs)
        resp = self.__k8s_beta.delete_namespaced_deployment(name, namespace, body, **kwargs)
        return resp
    def deleteAllDeployment(self, namespace="default", **kwargs):
        respd = self.__k8s_beta.delete_collection_namespaced_deployment(namespace)
        respr = self.__k8s_beta.delete_collection_namespaced_replica_set(namespace)
        respp = self.__k8s_core.delete_collection_namespaced_pod(namespace)
        return respd,respr,respp

    def createService(self, filename, filedir=None):
        if filedir is not None:
            filename = os.path.join(filedir, filename)
        with open(filename) as f:
            svc = yaml.load(f)            
            resp = self.__k8s_core.create_namespaced_service(
                body=svc, namespace="default")
            print("SERVICE {0} created. status={1}".format(filename, resp.metadata))
        return resp
    def getServiceList(self):
        resp = self.__k8s_core.list_service_for_all_namespaces()
        return resp
    def deleteService(self, name, namespace="default", **kwargs):
        resp = self.__k8s_core.delete_namespaced_service(name, namespace, **kwargs)
        return resp

In [15]:
kd = KubeDeployer()

In [16]:
kdeploy = kd.createDeployment("nginxdep.yml", "/opt/workdir/kube/")

DEPLOYMENT /opt/workdir/kube/nginxdep.yml created. status={'annotations': None,
 'cluster_name': None,
 'creation_timestamp': datetime.datetime(2018, 2, 4, 6, 50, 13, tzinfo=tzlocal()),
 'deletion_grace_period_seconds': None,
 'deletion_timestamp': None,
 'finalizers': None,
 'generate_name': None,
 'generation': 1,
 'initializers': None,
 'labels': {'run': 'my-nginx'},
 'name': 'my-nginx',
 'namespace': 'default',
 'owner_references': None,
 'resource_version': '14964',
 'self_link': '/apis/extensions/v1beta1/namespaces/default/deployments/my-nginx',
 'uid': 'a66f4390-0977-11e8-b962-42010a920004'}


In [17]:
ksvc = kd.createService("nginxsvc.yml", "/opt/workdir/kube/")

SERVICE /opt/workdir/kube/nginxsvc.yml created. status={'annotations': None,
 'cluster_name': None,
 'creation_timestamp': datetime.datetime(2018, 2, 4, 6, 50, 13, tzinfo=tzlocal()),
 'deletion_grace_period_seconds': None,
 'deletion_timestamp': None,
 'finalizers': None,
 'generate_name': None,
 'generation': None,
 'initializers': None,
 'labels': {'run': 'my-nginx'},
 'name': 'my-nginx',
 'namespace': 'default',
 'owner_references': None,
 'resource_version': '14981',
 'self_link': '/api/v1/namespaces/default/services/my-nginx',
 'uid': 'a6d7c669-0977-11e8-b962-42010a920004'}


In [18]:
kd.deleteAllDeployment()
kd.deleteService('my-nginx')

{'api_version': 'v1',
 'code': None,
 'details': None,
 'kind': 'Status',
 'message': None,
 'metadata': {'_continue': None, 'resource_version': None, 'self_link': None},
 'reason': None,
 'status': 'Success'}